In [ ]:
import numpy as np
import pandapower as pp
from deap import base, creator, tools, algorithms
import random
import time

# === 기본 설정 ===
S_base = 100      # MVA
V_base_kv = 110   # kV (데이터 시트 확인 필요)
Z_base = V_base_kv**2 / S_base if S_base != 0 else 0
f_hz = 60         # 주파수 60Hz 로 설정

# === IEEE 14-bus 시스템 데이터 템플릿 (기존과 동일) ===
bus_data_template = np.array([
    # ... (이전 코드와 동일하게 유지) ...
    [1,  0.0,        0.0,       1.05, 0.0],       # Bus 1 (Slack)
    [2, -21.7/100,   0.0,       1.05,  None],    # Bus 2 (PV)
    [2, -94.2/100,   0.0,       1.05,  None],    # Bus 3 (PV)
    [3, -47.8/100,   -(-3.9)/100, None, None],    # Bus 4 (PQ)
    [3, -7.6/100,    -0.6/100,    None, None],    # Bus 5 (PQ)
    [2, -11.2/100,   0.0,       1.05,  None],    # Bus 6 (PV)
    [3,  0.0,        0.0,       None, None],    # Bus 7 (PQ - Zero Load)
    [3,  0.0,        0.0,       None, None],    # Bus 8 (PQ - Zero Load)
    [3, -29.5/100,   -3.6/100,    None, None],    # Bus 9 (PQ)
    [3, -9/100,      -1.5/100,    None, None],    # Bus 10 (PQ)
    [3, -3.5/100,    -0.1/100,    None, None],    # Bus 11 (PQ)
    [3, -6.1/100,    -0.6/100,    None, None],    # Bus 12 (PQ)
    [3, None,       -2.8/100,    None, None],    # Bus 13 (PQ - Variable Load x0)
    [3, None,       -1.0/100,    None, None],    # Bus 14 (PQ - Variable Load x1)
], dtype=object)

branch_data = np.array([
    # ... (이전 코드와 동일하게 유지, 단, index 확인 필요) ...
    [1, 2, 0.01938, 0.05917, 0.0528], [1, 5, 0.05403, 0.22304, 0.0492],
    [2, 3, 0.04699, 0.19797, 0.0438], [2, 4, 0.05811, 0.17632, 0.0340],
    [2, 5, 0.05695, 0.17388, 0.0346], [3, 4, 0.06701, 0.17103, 0.0128],
    [4, 5, 0.01335, 0.04211, 0.0   ], [4, 7, 0.0,     0.20912, 0.0   ],
    [4, 9, 0.0,     0.55618, 0.0   ], [5, 6, 0.0,     0.25202, 0.0   ],
    [6,11, 0.09498, 0.19890, 0.0   ], [6,12, 0.12291, 0.25581, 0.0   ],
    [6,13, 0.06615, 0.13027, 0.0   ], [7, 8, 0.0,     0.17615, 0.0   ], # Bus 8 존재 확인
    [7, 9, 0.0,     0.11001, 0.0   ], [9,10, 0.03181, 0.08450, 0.0   ],
    [9,14, 0.12711, 0.27038, 0.0   ], [10,11,0.08205, 0.19207, 0.0   ],
    [12,13,0.22092, 0.19988, 0.0   ], [13,14,0.17093, 0.34802, 0.0   ],
], dtype=object)

# --- 시간별 부하 계수 (기존과 동일) ---
hourly_load_factors = np.array([
    0.4, 0.35, 0.3, 0.3, 0.35, 0.45, 0.6, 0.75, 0.85, 0.9, 0.95, 1.0,
    0.95, 0.9, 0.85, 0.8, 0.8, 0.85, 0.95, 1.0, 0.9, 0.8, 0.7, 0.5
])

# --- 네트워크 생성 함수 (주파수 반영) ---
def build_network(x0_h, x1_h, hour, load_profile):
    """주어진 시간(hour)과 해당 시간의 x0, x1 값, 부하 프로파일에 맞춰 네트워크 구성"""
    bus_data = bus_data_template.copy()
    net = pp.create_empty_network(name=f"IEEE 14 Bus - Hour {hour}")
    current_load_factor = load_profile[hour]

    for i, b in enumerate(bus_data):
        pp.create_bus(net, vn_kv=V_base_kv, name=f"Bus {i+1}", index=i)

    for i, b in enumerate(bus_data):
        typ, Ppu, Qpu, Vsp, va_deg = b
        bus_idx = i
        if typ == 1:
             pp.create_ext_grid(net, bus=bus_idx, vm_pu=Vsp if Vsp is not None else 1.0,
                               va_degree=va_deg if va_deg is not None else 0.0)
        elif typ == 2:
            Pmw = -Ppu * S_base if Ppu is not None else 0.0
            pp.create_gen(net, bus=bus_idx, p_mw=Pmw, vm_pu=Vsp if Vsp is not None else 1.0,
                          min_q_mvar=-100, max_q_mvar=100, controllable=False)
        elif typ == 3:
            base_Pmw = -Ppu * S_base if Ppu is not None else 0.0
            base_Qmvar = -Qpu * S_base if Qpu is not None else 0.0
            if i == 12: # Bus 13
                Pmw = x0_h # 해당 시간의 x0 값 사용
                Qmvar = base_Qmvar
            elif i == 13: # Bus 14
                Pmw = x1_h # 해당 시간의 x1 값 사용
                Qmvar = base_Qmvar
            else:
                Pmw = base_Pmw * current_load_factor
                Qmvar = base_Qmvar * current_load_factor
            if Pmw != 0 or Qmvar != 0:
                pp.create_load(net, bus=bus_idx, p_mw=Pmw, q_mvar=Qmvar)

    for f, t, R, X, Bpu in branch_data:
        if Z_base == 0: continue
        r_ohm = R * Z_base
        x_ohm = X * Z_base
        b_actual = Bpu / Z_base
        # === 주파수 60Hz 반영 ===
        c_nf_per_km = b_actual / (2 * np.pi * f_hz * Z_base) * 1e9 if (Z_base != 0 and Bpu != 0) else 0
        # ========================
        from_bus_idx = int(f-1)
        to_bus_idx = int(t-1)
        if from_bus_idx in net.bus.index and to_bus_idx in net.bus.index:
            try:
                pp.create_line_from_parameters(net, from_bus=from_bus_idx, to_bus=to_bus_idx, length_km=1.0,
                                                r_ohm_per_km=r_ohm, x_ohm_per_km=x_ohm,
                                                c_nf_per_km=c_nf_per_km, max_i_ka=10.0)
            except Exception:
                pass
    return net

# --- GA 설정 (개체 크기 48, 최대화 목표) ---
if "FitnessMax" in creator.__dict__:
    del creator.FitnessMax
if "Individual" in creator.__dict__:
    del creator.Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # 최대화
# 개체는 48개의 float 리스트 (x0_h0, x1_h0, x0_h1, x1_h1, ..., x0_h23, x1_h23)
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# 시간별 x0, x1 값의 범위 설정 (MW 단위) - 모든 시간에 동일 범위 적용
HOURLY_VAR_MIN = 0
HOURLY_VAR_MAX = 50 # 예시 범위 (필요시 조정)
toolbox.register("attr_hourly_val", random.uniform, HOURLY_VAR_MIN, HOURLY_VAR_MAX)

# 개체 생성 함수: 48개의 랜덤 값으로 초기화
N_VARS = 48 # 2 variables * 24 hours
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_hourly_val, n=N_VARS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# --- 평가 함수 수정 (시간별 x0, x1 사용, x0 총합 최대화) ---
def evaluate_24hr_variable_x0sum(individual, load_profile):
    """시간별 x0, x1 값을 사용하는 24시간 시뮬레이션 평가. x0 총합을 최대화."""
    if len(individual) != N_VARS:
         raise ValueError(f"Individual length ({len(individual)}) does not match expected {N_VARS}")

    total_x0_sum = 0 # 하루 동안의 x0 합계

    for hour in range(24):
        # 해당 시간의 x0, x1 값 추출
        x0_h = individual[hour * 2]
        x1_h = individual[hour * 2 + 1]

        net = build_network(x0_h, x1_h, hour, load_profile)
        try:
            pp.runpp(net, algorithm='nr', calculate_voltage_angles=True, init="flat", tolerance_mva=1e-6, max_iteration=30)
            vm = net.res_bus.vm_pu.values
            # 슬랙 버스(index 0) 제외하고 전압 확인
            if np.any(vm[1:] < 1.0) or np.any(vm[1:] > 1.05):
                return (-float('inf'),) # 전압 제약 위반

            # (선택) 선로 과부하 제약 추가 가능
            # loading = net.res_line.loading_percent.values
            # if np.any(loading > 100.0):
            #     return (-float('inf'),)

        except pp.LoadflowNotConverged:
            return (-float('inf'),) # 조류 계산 실패
        except Exception as e:
            # print(f"Hour {hour} evaluation error: {e}") # 디버깅
            return (-float('inf'),) # 기타 예외

    # 모든 시간 제약 만족 시: x0의 총합을 적합도로 반환
    total_x0_sum = sum(individual[h * 2] for h in range(24))
    return (total_x0_sum,)

# Toolbox에 새로운 평가 함수 등록
toolbox.register("evaluate", evaluate_24hr_variable_x0sum, load_profile=hourly_load_factors)
toolbox.register("mate", tools.cxBlend, alpha=0.5) # 또는 cxTwoPoint 등
# 돌연변이: 48개 유전자에 대해 적용, sigma 값 조정 필요
MUT_SIGMA = 5 # 시간별 변동폭 고려하여 조정
toolbox.register("mutate", tools.mutGaussian, mu=[0]*N_VARS, sigma=[MUT_SIGMA]*N_VARS, indpb=0.1) # indpb 확률 조정 가능
toolbox.register("select", tools.selTournament, tournsize=3)

# --- 실행 함수 ---
def main_24hr_variable():
    start_run_time = time.time()

    # 개체군 크기 및 세대 수 (계산 시간 매우 오래 걸림! 대폭 축소)
    pop_size = 50  # 늘려야 할 수도 있음 (탐색 공간이 커짐)
    num_generations = 30 # 늘려야 할 수도 있음

    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0]) # 단일 목표
    stats.register("max", np.max) # x0 총합 최대화
    stats.register("avg", np.mean)

    print(f"Starting GA for 24-hour variable load simulation (Pop: {pop_size}, Gen: {num_generations})...")
    print("Objective: Maximize sum of hourly x0 values.")
    print("This will take a VERY long time.")

    # 유전 알고리즘 실행
    pop, log = algorithms.eaSimple(pop, toolbox,
                                   cxpb=0.7, mutpb=0.2, # 교차/변이 확률 조정
                                   ngen=num_generations, stats=stats,
                                   halloffame=hof, verbose=True)

    end_run_time = time.time()
    print(f"\nGA execution time: {end_run_time - start_run_time:.2f} seconds.")

    # 최적해 출력
    if hof:
        best_individual = hof[0]
        best_fitness = best_individual.fitness.values[0]

        if best_fitness > -float('inf'):
            print("\n--- Optimal Solution Found (Hourly Variable Loads) ---")
            print(f"Maximum Total Daily Sum of x0: {best_fitness:.4f}")

            print("\nOptimal Hourly Schedule (x0, x1) in MW:")
            print("Hour |   x0   |   x1   ")
            print("-----|--------|--------")
            hourly_x0 = [best_individual[h * 2] for h in range(24)]
            hourly_x1 = [best_individual[h * 2 + 1] for h in range(24)]
            for h in range(24):
                print(f"{h:4d} | {hourly_x0[h]:6.2f} | {hourly_x1[h]:6.2f}")

            # 피크 시간대 검증
            peak_hour = np.argmax(hourly_load_factors)
            print(f"\nVerifying results for peak hour ({peak_hour}):")
            peak_x0 = hourly_x0[peak_hour]
            peak_x1 = hourly_x1[peak_hour]
            final_net = build_network(peak_x0, peak_x1, peak_hour, hourly_load_factors)
            try:
                pp.runpp(final_net, algorithm='nr', calculate_voltage_angles=True, init="flat")
                print("  Power flow converged for peak hour with optimal schedule.")
                print("\n=== Peak Hour Bus Results (res_bus) ===")
                print(final_net.res_bus[['vm_pu', 'va_degree', 'p_mw', 'q_mvar']])
                # print("\n=== Peak Hour Line Results (res_line) ===")
                # print(final_net.res_line[['p_from_mw', 'q_from_mvar', 'loading_percent']])
            except Exception as e:
                print(f"  Error during final verification for peak hour: {e}")
        else:
            print("\nNo feasible hourly schedule found that satisfies constraints for all 24 hours.")
            print("Try increasing population/generations, adjusting mutation/crossover rates,")
            print("or check variable ranges and constraints.")
    else:
        print("\nHall of Fame is empty. No solution found.")

# 실행
if __name__ == "__main__":
    main_24hr_variable()